In [1]:
from read_datasetBreakfast import load_data, read_mapping_dict, load_data_cust
import os
import torch
import seaborn as sn

COMP_PATH = 'C:\\Users\\cherl\\Desktop\\Masters\\CS5242 Neural Networks and Deep Learning\\Project'
''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

actions_dict = read_mapping_dict(mapping_loc)
if  split == 'training':
    data_feat_train, data_labels = load_data(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
    data_feat_train_cust, data_labels_cust = load_data_cust(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
if  split == 'test':
    data_feat_test = load_data(test_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features only

#'''
#Write Code Below
#Pointers
#Need to load the segments.txt file for segments for test videos 
#Output the CSV in correct format as shown in Evaluation Section
#Id corresponds to the segments in order. 
#Example - 30-150 = Id 0
#          150-428 = Id 1
#          428-575 = Id 2
#Category is the Class of the Predicted Action
#'''

Finish Load the Training data and labels!!!
Finish Load the Training data and labels!!!


In [2]:
#Print info on the data
print("Number of Training Data: {}".format(len(data_feat_train)))
print("Number of Training Data Labels: {}".format(len(data_labels)))
print("Size of Training Data #0: {}".format(data_feat_train[0].size()))
#print(data_feat_train[0][0])
print(len(data_labels_cust[0]))
#Testing Data
try:
    print("Number of Test Data: {}".format(len(data_feat_test)))
    print("% of Training Data: {}".format(len(data_feat_test)/(len(data_feat_train)+len(data_feat_test))*100))
except NameError:
    print("Testing Data not loaded")

Number of Training Data: 28
Number of Training Data Labels: 28
Size of Training Data #0: torch.Size([743, 400])
743
Testing Data not loaded


In [3]:
#Preparing the data by splitting video into segments
segments = []
segment_labels = []
for i, video in enumerate(data_feat_train[0:1]):
    frame_num = 0
    while frame_num < len(video):
        #Skip SIL frames
        if data_labels_cust[i][frame_num] > 0:
            start_index = frame_num # start index of segment
            end_index = frame_num # end index of segment
            start_label = data_labels_cust[i][frame_num] # label of segment
            for index in range(start_index + 1, len(video)):
                if data_labels_cust[i][index] == start_label:
                    frame_num=index + 1
                    end_index=index
                else:
                    frame_num=index + 1
                    break
            segment = video[start_index:end_index]
            segments.append(segment)
            segment_label = []
            segment_label.append(start_label)
            segment_labels.append(segment_label)
            #print(segment.size())
            #print("start {}, end {}".format(start_index, end_index))
        else:
            frame_num += 1
print(segment_labels)

#Making all the segments equal length
standardized_segments = []
target_segment_length = 300 # Standard length to convert all segments to. Tune as needed.

for segment in segments:
    if len(segment) > target_segment_length:
        # Truncate segment
        standardized_segment = segment[0:target_segment_length]
        standardized_segments.append(standardized_segment)
    elif len(segment) == target_segment_length:
        standardized_segment = segment
        standardized_segments.append(standardized_segment)
    else:
        # Pad segment
        standardized_segment = torch.zeros(target_segment_length, 400)
        standardized_segment[0:len(segment)] = segment
        standardized_segments.append(standardized_segment)
    #print(standardized_segment.size())

[[1], [2], [3]]


In [17]:
import torch.nn as nn
import torch.nn.functional as F

#Define the neural network modules

class mynet(nn.Module):
    def __init__(self):
        super(mynet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 10, stride=5).double()
        self.pool = nn.MaxPool2d(5, stride=5).double()
        self.fc1 = nn.Linear(target_segment_length * 16, 47).double()
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

class myGruNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(myGruNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    #Initialize the hidden layer
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class myRnn(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size, n_layers=1):
        super(myRnn, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, input):
         
        batch_size = input.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(input, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)

In [85]:
# Helper functions to run the training
def cal_acc(probs, target):
    # probs: probability that each image is labeled as 1
    # target: ground truth label
    with torch.no_grad():
      prediction = torch.argmax(probs, axis=-1)    
      acc = torch.sum(prediction == target)
    return acc.item() / len(target) * 100

def train_one_pass_rnn(network, optim, device, criterion):
  # Input: Network and Optimizer
  # Output: Averge accuracy , Avergae loss in the pass
  network.train()
  acc_one_pass = []
  loss_one_pass = []
  for i, segment in enumerate(standardized_segments):
    X = torch.tensor(segment).unsqueeze(dim=0).float().to(device)
    y = torch.ones(X.size()[1])
    y = torch.Tensor.new_full(y, y.size(), segment_labels[i][0], dtype=int, device=device, requires_grad=True)
    out, h = network(X)
    print(out)
    print(y)
    loss = criterion(out, y)
    
#     for frame in segment:
#         X = torch.tensor(frame).float().to(device)
#         h = network.init_hidden().to(device)
#         y = torch.tensor(segment_labels[i]).to(device)
#         out, h = network(X, h)
#         #probs = F.softmax(out, dim=1)
#         #predy = torch.argmax(probs, dim=1)
#         loss = criterion(out, y)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if i % 1 == 0:
          print("Iter: {:.2f}. ".format(i / len(standardized_segments)),
            "Train Total Loss: {:.2f}. ".format(loss.item()),
            "Train Accuracy: {:.2f}% ".format(cal_acc(out, y))
          )
    acc_one_pass.append(cal_acc(out, y))
    loss_one_pass.append(loss.item())
    return sum(acc_one_pass) / len(acc_one_pass), sum(loss_one_pass) / len(loss_one_pass)

IndentationError: unexpected indent (<ipython-input-85-4881e12b1b1a>, line 38)

In [86]:
#Run the training

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

# Model paramters
input_dim = 400
hidden_dim = 256
output_dim = 47
n_layers = 2
    
#net = myGruNet(input_dim, hidden_dim, output_dim, n_layers).to(device)
net = myRnn(input_dim, hidden_dim, output_dim).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()
    
avg_acc, avg_loss = train_one_pass_rnn(net, optimizer, device, criterion)

C:\Users\cherl\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor([[ 0.5887, -0.0030, -0.1219,  ...,  0.5042, -0.5939, -0.3159],
        [ 0.4351,  0.2384,  0.0957,  ...,  0.5670, -0.6630,  0.0778],
        [ 0.5049,  0.2067, -0.0947,  ...,  0.7911, -0.4287, -0.1147],
        ...,
        [ 0.6918, -0.8109, -0.1460,  ...,  0.1309,  0.0309,  0.7485],
        [ 0.6670, -0.5091, -0.1392,  ..., -0.0219, -0.0778,  0.6554],
        [ 0.6574, -0.7828,  0.1567,  ..., -0.0887, -0.2390,  0.5096]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1,